# Relevant Imports

In [1]:
import numpy as np
import pandas as pd
import glob
import os
import xgboost
import csv as csv
import ipychart as ipc
from xgboost import plot_importance
from matplotlib import pyplot
from sklearn.model_selection import cross_val_score,KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
# from sklearn.grid_search import GridSearchCV   #Perforing grid search
from scipy.stats import skew
from collections import OrderedDict

In [2]:
dataFrame = pd.read_csv("./CombinedData.csv", header = 0, low_memory=False)
dataFrame.insert(0, 'ID', range(0,len(dataFrame)))
dataFrame = dataFrame.sample(frac=1).reset_index(drop=True)
print(dataFrame.head())

      ID          SOLD DATE              PROPERTY TYPE              ADDRESS  \
0  40840    October-27-2021  Single Family Residential   9810 Sweet Plum Dr   
1  56037                NaN  Single Family Residential        327 Foster St   
2   8771  September-23-2021  Single Family Residential   2114 Grist Mill Dr   
3  54453                NaN  Single Family Residential       335 Watkins St   
4   3073   November-10-2021                Condo/Co-op  5713 Bramblegate Rd   

         CITY STATE OR PROVINCE ZIP OR POSTAL CODE   PRICE  BEDS  BATHS  \
0   Charlotte                NC              28105  349000   3.0    2.5   
1  Burlington                NC              27244   72500   3.0    1.0   
2     Concord                NC              28025  570000   6.0    5.0   
3    Asheboro                NC              27370  105000   NaN    1.5   
4  Greensboro                NC              27409  121000   4.0    3.0   

                   LOCATION  SQUARE FEET  LOT SIZE  YEAR BUILT  $/SQUARE F

In [3]:
dataFrame = dataFrame[dataFrame['SQUARE FEET'].notna()]
dataFrame = dataFrame[dataFrame['SOLD DATE'].notna()]
dataFrame = dataFrame[dataFrame['ZIP OR POSTAL CODE'].notna()]
dataFrame = dataFrame[dataFrame['YEAR BUILT'].notna()]

# Normalizing DataFrame Data

In [4]:
dataFrame['ZIP OR POSTAL CODE'] = pd.Series([int(x.split('-')[0].split(' ')[0]) if '-' in x or ' ' in x else int(x) for x in dataFrame['ZIP OR POSTAL CODE']])
# dataFrame['ZIP OR POSTAL CODE'] = dataFrame['ZIP OR POSTAL CODE'].astype(int)
df = dataFrame.copy(deep=True)
dataFrame = dataFrame.drop(columns=["URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)", 
                                    "LOCATION", 'PROPERTY TYPE', 'CITY', 'STATE OR PROVINCE', 'ADDRESS'])
df = df.drop(columns=["URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)", 
                                    "LOCATION", 'PROPERTY TYPE', 'CITY', '$/SQUARE FEET'])

In [5]:
dataFrame = dataFrame[dataFrame['SQUARE FEET'].notna()]
dataFrame = dataFrame[dataFrame['SOLD DATE'].notna()]
dataFrame = dataFrame[dataFrame['ZIP OR POSTAL CODE'].notna()]
dataFrame = dataFrame[dataFrame['YEAR BUILT'].notna()]

In [6]:
dataFrame['SOLD DATE'] = pd.to_datetime(dataFrame['SOLD DATE']).apply(lambda x: x.value)

In [7]:
# newColumns = ['ID',
# 'SOLD DATE',
# 'PROPERTY TYPE',
# 'ADDRESS',
# 'CITY',
# 'STATE OR PROVINCE',
# 'ZIP OR POSTAL CODE',
# 'BEDS',
# 'BATHS',
# 'SQUARE FEET',
# 'LOT SIZE',
# 'YEAR BUILT',
# '$/SQUARE FEET',
# 'HOA/MONTH',
# 'LATITUDE',
# 'LONGITUDE',
# 'PRICE']
# dataFrame = dataFrame[newColumns]

In [8]:
category_features = []
labels = dataFrame['PRICE']
training = dataFrame.drop(['ID'], axis = 1)

# Manipulating non-normalized dataframe

In [9]:
df['SOLD DATE'] = pd.to_datetime(df['SOLD DATE'])

In [10]:
print(df.set_index('SOLD DATE').loc['2022-3-1':'2022-6-1'])

               ID                  ADDRESS STATE OR PROVINCE  \
SOLD DATE                                                      
2022-03-04  19937           420 Woodson Rd                NC   
2022-03-04  21990    15655 Cool Springs Rd                NC   
2022-04-14  33748          26 Shadymont Dr                NC   
2022-05-24  40646         13507 Kintyre Ct                NC   
2022-04-19  63083     4628 Orange Grove Rd                NC   
...           ...                      ...               ...   
2022-04-14  15332            110 Watson St                NC   
2022-03-28  73419  1365 Amberlight Cir #42                NC   
2022-04-05  48571            38 Chapman Ct                NC   
2022-05-27  37977      923 Country Club Dr                NC   
2022-03-11  49063      448 Martins Mill Ct                NC   

            ZIP OR POSTAL CODE   PRICE  BEDS  BATHS  SQUARE FEET  LOT SIZE  \
SOLD DATE                                                                    
2022-03-04 

In [11]:
df.columns

Index(['ID', 'SOLD DATE', 'ADDRESS', 'STATE OR PROVINCE', 'ZIP OR POSTAL CODE',
       'PRICE', 'BEDS', 'BATHS', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT',
       'HOA/MONTH', 'LATITUDE', 'LONGITUDE'],
      dtype='object')

In [12]:
chart = ipc.lineplot(data=df, x='SOLD DATE', y = 'PRICE', hue='')
chart

Chart(layout=Layout(align_self='stretch', height='auto'))

In [13]:
dfView = df.set_index('SOLD DATE',drop=False).loc['2022-3-1':'2022-6-1'].sort_values('YEAR BUILT', ascending=False)
indexSeries = pd.Series(dfView.index.tolist())

In [14]:
chart2 = ipc.lineplot(data = dfView, x= 'YEAR BUILT', y= 'PRICE', hue='')
chart2

Chart(layout=Layout(align_self='stretch', height='auto'))

In [15]:
# chart.to_html('./chartExample')

In [16]:
X_train, X_test, y_train, y_test = train_test_split(training, labels, test_size=.3)
train_dataset = pd.get_dummies(X_train,columns=category_features)
test_dataset = pd.get_dummies(X_test,columns=category_features)
every_column_except_y= [col for col in train_dataset.columns if col not in ['PRICE','ID']]
train_X = train_dataset[every_column_except_y]
every_column_except_y= [col for col in test_dataset.columns if col not in ['PRICE','ID']]
test_X = test_dataset[every_column_except_y]
train_Y = train_dataset['PRICE']
test_Y = test_dataset['PRICE']

In [17]:
non_categorical_columns = [col for col in train_X.columns if col not in category_features and col not in ['ID']]
numeric_features = train_X[non_categorical_columns].dtypes[train_X.dtypes != "object"].index
train_X[numeric_features] = np.log1p(train_X[numeric_features])

non_categorical_columns = [col for col in test_X.columns if col not in category_features and col not in ['ID']]
numeric_features = test_X[non_categorical_columns].dtypes[test_X.dtypes != "object"].index
test_X[numeric_features] = np.log1p(test_X[numeric_features])

C:\Users\Joey\AppData\Local\Temp/ipykernel_13256/1742185009.py:3: RuntimeWarning: invalid value encountered in log1p
  train_X[numeric_features] = np.log1p(train_X[numeric_features])
C:\Users\Joey\anaconda3\lib\site-packages\pandas\core\frame.py:3069: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]
C:\Users\Joey\AppData\Local\Temp/ipykernel_13256/1742185009.py:7: RuntimeWarning: invalid value encountered in log1p
  test_X[numeric_features] = np.log1p(test_X[numeric_features])


In [18]:
# nan_features = ['LOCATION']
# def ConvertToNAString(data, columnsList):
#     for x in columnsList:
#         data[x] = str(data[x])
# ConvertToNAString(train_dataset, nan_features)
# ConvertToNAString(test_dataset, nan_features)
X_train.columns

Index(['SOLD DATE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'HOA/MONTH', 'LATITUDE',
       'LONGITUDE'],
      dtype='object')

In [19]:
def createColumnPerValue(data, columnsList):
  for x in columnsList:
    values = pd.unique(data[x])

    for y in values: 
      column_name = x + "_" + str(y)
      data[column_name]=(data[x]==y).astype(float)
    
    data.drop(x, axis=1, inplace=True)

# Establish ML model

In [20]:
 model = xgboost.XGBRegressor(colsample_bytree=0.4,
                 gamma=0,                 
                 learning_rate=0.07,
                 max_depth=3,
                 tree_method='gpu_hist', 
                 gpu_id=0,
                 min_child_weight=1.5,
                 n_estimators=10000,                                                                    
                 reg_alpha=0.75,
                 reg_lambda=0.45,
                 subsample=0.6,
                 seed=42) 
# model = xgboost.XGBRegressor()
# model.load_model("CurrentModel.json")

In [21]:
#df['time'] = df['time'].apply(lambda x: x.value)
print(train_dataset.columns)

Index(['SOLD DATE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'HOA/MONTH', 'LATITUDE',
       'LONGITUDE'],
      dtype='object')


In [22]:
print(train_X.columns)

Index(['SOLD DATE', 'ZIP OR POSTAL CODE', 'BEDS', 'BATHS', 'SQUARE FEET',
       'LOT SIZE', 'YEAR BUILT', 'HOA/MONTH', 'LATITUDE', 'LONGITUDE'],
      dtype='object')


In [23]:
# every_column_except_y= [col for col in X_train.columns if col not in ['PRICE','ID']]
model.fit(train_X, train_Y)
OrderedDict(sorted(model.get_booster().get_fscore().items(), key=lambda t: t[1], reverse=True))

OrderedDict([('SQUARE FEET', 10593.0),
             ('LOT SIZE', 9816.0),
             ('LATITUDE', 9458.0),
             ('YEAR BUILT', 8400.0),
             ('SOLD DATE', 8194.0),
             ('ZIP OR POSTAL CODE', 7360.0),
             ('HOA/MONTH', 6083.0),
             ('BATHS', 5472.0),
             ('BEDS', 3337.0)])

In [24]:
model.save_model("CurrentModel.json")

In [25]:
most_relevant_features= list( dict((k, v) for k, v in model.get_booster().get_fscore().items() if v >= 10).keys())
print(most_relevant_features)

['SOLD DATE', 'ZIP OR POSTAL CODE', 'BEDS', 'BATHS', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'HOA/MONTH', 'LATITUDE']


In [26]:
y_pred = model.predict(test_X)
predictions = [round(value) for value in y_pred]
print(predictions)

[321560, 283960, 237623, 294416, 332189, 434852, 522037, 677312, 356481, 294376, 461450, 416719, 558396, 493991, 292854, 445137, 522784, 474923, 247227, 481478, 324376, 336116, 551680, 216208, 429390, 186508, 273309, 299830, 281594, 332849, 409048, 381095, 274769, 298118, 372540, 581122, 369472, 423551, 294220, 336125, 516440, 363865, 257155, 302673, 243362, 386148, 1051685, 289329, 622598, 374684, 400935, 383053, 320135, 426710, 262195, 360919, 388498, 939472, 133236, 524030, 384999, 323575, 623709, 333464, 434997, 614908, 689077, 310564, 486747, 355484, 248122, 432381, 281062, 383206, 192391, 350776, 367690, 441589, 305731, 2665557, 437222, 299346, 436658, 649253, 1176491, 222580, 271584, 307189, 480098, 219842, 228984, 359250, 385419, 135000, 446007, 260756, 766982, 514801, 676191, 240260, 595290, 244578, 319467, 420580, 504442, 356887, 362028, 1212439, 1161490, 472252, 312586, 222479, 228665, 506906, 199182, 333078, 267202, 310730, 251912, 232244, 286337, 406191, 400230, 451261, 16

In [27]:
accuracy = accuracy_score(test_Y, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 0.00%


In [28]:
test_Y.head(100)

27109     250000
12716     240000
7         310000
11088     341000
9950      327420
          ...   
40332     240000
28591    1475000
28167     530000
10857     700000
33674     352500
Name: PRICE, Length: 100, dtype: int64

In [29]:
errlist = []
ty = list(test_Y)
for i, v in enumerate(predictions):
    errlist.append(abs(ty[i]-predictions[i]))

sum(errlist)/len(errlist)

84114.77291325695

In [30]:
errlist

[71560,
 43960,
 72377,
 46584,
 4769,
 15497,
 62963,
 41672,
 20991,
 55624,
 67830,
 48281,
 174604,
 48029,
 97146,
 20237,
 56216,
 249877,
 11227,
 4522,
 45624,
 56116,
 53231,
 16208,
 59390,
 50492,
 36691,
 44830,
 23405,
 2359,
 44048,
 4095,
 49279,
 58382,
 102460,
 103782,
 26272,
 8611,
 74220,
 18875,
 43470,
 13865,
 32845,
 35173,
 48362,
 11148,
 281685,
 51571,
 132598,
 1316,
 7660,
 44947,
 179865,
 148290,
 74205,
 6419,
 60655,
 297720,
 16764,
 23970,
 249999,
 82675,
 153945,
 38464,
 4317,
 177908,
 20868,
 22074,
 753253,
 14516,
 28122,
 12381,
 41062,
 111794,
 27391,
 19276,
 69810,
 28689,
 110731,
 1634443,
 67922,
 14346,
 4802,
 30224,
 336491,
 42420,
 60916,
 112811,
 137902,
 50158,
 256016,
 125750,
 10419,
 35000,
 67993,
 20756,
 708018,
 15199,
 23809,
 112240,
 195710,
 59578,
 39467,
 84580,
 156442,
 41887,
 81695,
 237439,
 238510,
 39364,
 1586,
 64979,
 60435,
 85094,
 9182,
 60078,
 147202,
 21430,
 18088,
 102756,
 28663,
 6191,
 35330,